In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

Start by grabbing our webpage

In [2]:
urls = ['http://www.city-data.com/nbmaps/neigh-Atlanta-Georgia.html', 'http://www.city-data.com/nbmaps/neigh-Atlanta-Georgia2.html']
html_data = []
for i in urls:
    page = requests.get(i)
    soup = BeautifulSoup(page.text, 'html.parser')
    html_data += soup.find_all('div', class_='neighborhood')

Everything below is essentially preprocessing. The data is not laid out nicely for us so we have to prepare it to be processed.

In [3]:
data = [i.text.replace(',', '').split('\n') for i in html_data]
x = list(data)
for i in range(0, len(data)):
    data[i] = data[i][:2]
    data[i][0] = data[i][0][:data[i][0].find(' neighborhood')]
    subdat = re.split("%\)(?!.*%\))", data[i][1])
    data[i][1:] = re.split(r'Most ', subdat[0])[:3]
    if len(data[i]) < 4:
        data[i] = None
newdat = []
for x in data:
    if x:
        newdat.append(x)
data = list(newdat)

Now we can use more regular expressions to process our data and generate our columns.

In [4]:
atl_data = [['Neighborhood', 'Area', 'Population', 'Median Income', 'Median Rent', 'Num Males', 'Num Females', 'Median Age Male', 'Median Age Female']]
for i in data:
    row = [i[0]]
    row.append(re.search(r'(?<=Area: )[0-9.]*', i[1])[0])
    row.append(re.search(r'(?<=Population: )[0-9.]*', i[1])[0])
    income = re.search(r'(?<=Median household income in 2016: ' + i[0] + r':\$)[0-9\.]*', i[1])
    if income:
        income = income[0]
    rent = re.search(r'(?<=Median rent in in 2016: ' + i[0] + r':\$)[0-9\.]*', i[1])
    if rent:
        rent = rent[0]
    male = re.findall(r'(?<=Males:)[0-9.]*', i[1])
    num_male = None
    num_female = None
    age_male = None
    age_female = None
    if len(male) >= 2:
        num_male = male[0]
        age_male = male[1]
    female = re.findall(r'(?<=Females:)[0-9.]*', i[1])
    if len(female) >= 2: 
        num_female = female[0]
        age_female = female[1]
    row += [income, rent, num_male, num_female, age_male, age_female]
    atl_data.append(row)

Load the data into a dataframe.

In [5]:
df = pd.DataFrame(columns=atl_data[0], data=atl_data[1:])

Everything looks pretty good. We'll have to decide what to do about NaNs later.

In [6]:
df.head(10)

,Neighborhood,Area,Population,Median Income,Median Rent,Num Males,Num Females,Median Age Male,Median Age Female
0,26th Street/Ardmore,0.070,341,66548,1113,170,170,31.5,29.3
1,Aberdeen Forest,0.322,705,82962,1259,274,430,36.3,37.4
2,Adair Park,0.375,1645,24874,626,760,885,34.2,39.7
3,Adamsville,0.995,3734,24372,593,1407,2326,20.6,33.2
4,Amberidge,0.177,313,137237,2581,147,166,52.1,49.0
5,Ansley Park,0.589,2188,81342,1097,1030,1158,40.8,40.1
6,Arden Habersham,0.220,242,250001,1100,143,99,40.7,49.7
7,Arrowood Drive Neighbourhood,0.188,512,None,None,197,314,36.6,43.8
8,Ashford Park,1.677,2928,75381,1042,1628,1299,33.5,33.5
9,Atkins Park,0.046,452,63580,796,232,219,39.3,36.5


Save csv to minimize requests.

In [7]:
df.to_csv('atl_neighborhoods.csv')